# Confusion Matrix

In [1]:
#Imports
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
from torchvision import datasets, models, transforms
from torch.utils.data import TensorDataset
import os
import random

In [2]:
#Master dataset
master = torchvision.datasets.ImageFolder('data_color_shifted', transform = transforms.ToTensor())

#split indexes
train_size = int(0.6 * len(master))
val_size = int(0.2 *len(master))
test_size = len(master) - val_size - train_size

train_set, val_set, test_set = torch.utils.data.random_split(master, [train_size, val_size, test_size])
print ("Length of training set: {}".format(len(train_set)))
print ("Length of val set: {}".format(len(val_set)))
print ("Length of test set: {}".format(len(test_set)))

Length of training set: 3485
Length of val set: 1161
Length of test set: 1163


In [3]:
def generate_CF_matrix(model, data_loader):
    '''
    Generate confusion matrix
    
    Model: The CNN Model
    
    data_loader: torch dataloader
    '''
    
    classes = ['INCORRECT_MASK', 'MASK', 'NO_MASK']
    CF_matrix = np.zeros((3, 4))
    
    for imgs, labels in data_loader:


        #############################################
        #To Enable GPU Usage
        if use_cuda and torch.cuda.is_available():
            imgs = imgs.cuda()
            labels = labels.cuda()
        #############################################


        output = model(imgs)

        #select index with maximum prediction score
        pred = output.max(1, keepdim=True)[1]
        for label, prediction in zip(labels, pred.view_as(labels)):
            CF_matrix[label][prediction] += 1
            
    for i in range(3):
        CF_matrix[i][3] = 100 * (sum(CF_matrix[i]) - CF_matrix[i][i]) / sum(CF_matrix[i])
            
            
    s = "\t\t\t\t\tPrediction\t\t\t\t\tError\n" +\
        "\t\t\t{}\t\t{}\t\t{}\n\n".format(*classes) +\
        "\t{}\t\t{}\t\t{}\t\t{}\t\t\t{:.4f}%\n\n".format(classes[0], *CF_matrix[0]) +\
        "Label\t{}\t\t\t{}\t\t{}\t\t{}\t\t\t{:.4f}%\n\n".format(classes[1], *CF_matrix[1]) +\
        "\t{}\t\t\t{}\t\t{}\t\t{}\t\t\t{:.4f}%\n\n".format(classes[2], *CF_matrix[2])
    
        
    return s

In [4]:
class CNN_V3(nn.Module):
    def __init__(self, num_classes=3):
        super(CNN_V3, self).__init__()
        self.name = "CNN_V3"
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=10, stride=4, padding=3)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=6, stride=1)
        
        self.drop = nn.Dropout2d(p=0.2)
        
        # 240x240x3 ==conv1=> 60x60x64 ==4x conv2=> 40x40x64
        self.fc = nn.Linear(in_features=40*40*64, out_features=num_classes)

    def forward(self, x):
        # Convolution with large kernel + stride to prevent need for max pooling (maintains accuracy and speeds up training)
        x = F.relu(self.conv1(x)) 
        
        # 64 feature convolutional layers
        x = F.relu(self.conv2(x)) 
        x = F.relu(self.conv2(x)) 
        x = F.relu(self.conv2(x)) 
        x = F.relu(self.conv2(x)) 

        # Dropout layer
        x = F.dropout(self.drop(x), training=self.training)
        
        x = x.view(-1, 40*40*64)
        x = self.fc(x)
        
        return x

In [5]:
use_cuda = True
torch.manual_seed(2020)

model_3 = CNN_V3()
model_3.load_state_dict(torch.load("models/CNN_3_lr=0.0003"))

if use_cuda and torch.cuda.is_available():
    model_3.cuda()
    
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=True)
test_CF_matrix = generate_CF_matrix(model_3, test_dataloader)
print(test_CF_matrix)

					Prediction					Error
			INCORRECT_MASK		MASK		NO_MASK

	INCORRECT_MASK		377.0		4.0		7.0			2.8351%

Label	MASK			4.0		377.0		4.0			2.0779%

	NO_MASK			6.0		2.0		382.0			2.0513%


